<a href="https://colab.research.google.com/github/Apyarn95/TransformersSequenceModelling/blob/master/Transformer_Advance_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#this is an advancement on basic transformer architecture . This architecture makes maximum use of the multiprocessing power of gpu and
import tensorflow as tf
import numpy as np
import unicodedata
import re
import time
import pandas as pd 
#this is to inform you are a piece of hsit

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/eng_french.csv',nrows=80000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,English words/sentences,French words/sentences,len_english,len_french,data_english,data_french_input,data_french_output
7714,I am a shy boy.,Je suis un garçon timide.,5,5,i am a shy boy .,<sostoken> je suis un garcon timide .,je suis un garcon timide . <eostoken>
7720,I am in a spot.,Je suis dans le pétrin.,5,5,i am in a spot .,<sostoken> je suis dans le petrin .,je suis dans le petrin . <eostoken>
7721,I am in a spot.,Je suis dans un endroit.,5,5,i am in a spot .,<sostoken> je suis dans un endroit .,je suis dans un endroit . <eostoken>
7883,I had to do it.,Il m'a fallu le faire.,5,5,i had to do it .,<sostoken> il m a fallu le faire .,il m a fallu le faire . <eostoken>
8135,I saw it on TV.,Je l'ai vu à la télé.,5,6,i saw it on tv .,<sostoken> je l ai vu a la tele .,je l ai vu a la tele . <eostoken>


In [3]:
data['len_english'] = data['English words/sentences'].apply(lambda x : len(x.split(" ")))
data['len_french'] = data['French words/sentences'].apply(lambda x : len(x.split(" ")))

In [4]:
data = data[data['len_english'] >= 5]
data = data[data['len_french'] >= 5]
data = data[data['len_english'] <= 14]
data = data[data['len_french'] <= 14]

In [5]:
#preprocessing
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD',s) 
                  if unicodedata.category(c)!= 'Mn')

def preprocess(s):
  s = s.lower()
  s = unicode_to_ascii(s)
  s = re.sub(r'([!.?])', r' \1', s)
  #creating space between word and punctuation following
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
  s = re.sub(r'\s+', r' ', s)
  return s



In [7]:
data['data_english'] = data['English words/sentences'].apply(lambda x:preprocess(x))
data['data_french_input'] = data['French words/sentences'].apply(lambda x : ('<sostoken> ' + preprocess(x)).strip())
data['data_french_output'] = data['French words/sentences'].apply(lambda x :  (preprocess(x).strip() + ' <eostoken>'))

In [8]:
# data_english , data_french = list(zip(*data))
# data_english , data_french = list(data_english) , list(data_french)
# data_english = [preprocess(data) for data in data_english]
# data_french_input = ['<sostoken> ' + preprocess(data) for data in data_french]
# data_french_output = [preprocess(data)+' <eostoken>' for data in data_french]
data_english = data['data_english'].tolist()
data_french_input = data['data_french_input'].tolist()
data_french_output = data['data_french_output'].tolist()


# Tokenization of sentences

In [9]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
en_tokenizer.fit_on_texts(data_english)
data_en = en_tokenizer.texts_to_sequences(data_english)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,
                                                        padding='post')

fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
fr_tokenizer.fit_on_texts(data_french_input)
fr_tokenizer.fit_on_texts(data_french_output)
data_fr_in = fr_tokenizer.texts_to_sequences(data_french_input)
data_fr_in = tf.keras.preprocessing.sequence.pad_sequences(data_fr_in,
                                                           padding='post')

data_fr_out = fr_tokenizer.texts_to_sequences(data_french_output)
data_fr_out = tf.keras.preprocessing.sequence.pad_sequences(data_fr_out,
                                                            padding='post')

385

# Creating tensorflow data object

In [10]:
BATCH_SIZE_ = 128
dataset = tf.data.Dataset.from_tensor_slices((data_en,data_fr_in,data_fr_out))
dataset = dataset.shuffle(20).batch(BATCH_SIZE_)

<BatchDataset shapes: ((None, 7), (None, 11), (None, 11)), types: (tf.int32, tf.int32, tf.int32)>


# Positional Embedding

In [11]:
def positional_embedding(pos, model_size):
    PE = np.zeros((1, model_size))
    for i in range(model_size):
        if i % 2 == 0:
            PE[:, i] = np.sin(pos / 10000 ** (i / model_size))
        else:
            PE[:, i] = np.cos(pos / 10000 ** ((i - 1) / model_size))
    return PE

max_length = max(len(data_en[0]), len(data_fr_in[0]))
MODEL_SIZE = 300

pes = []
for i in range(max_length):
    pes.append(positional_embedding(i, MODEL_SIZE))

pes = np.concatenate(pes, axis=0)
pes = tf.constant(pes, dtype=tf.float32)

# Mulithead attention

In [12]:
class MultiHeadAttention(tf.keras.Model):
    def __init__(self, model_size, h):
        super(MultiHeadAttention, self).__init__()
        self.key_size = model_size // h
        self.h = h
        self.wq = tf.keras.layers.Dense(model_size) #[tf.keras.layers.Dense(key_size) for _ in range(h)]
        self.wk = tf.keras.layers.Dense(model_size) #[tf.keras.layers.Dense(key_size) for _ in range(h)]
        self.wv = tf.keras.layers.Dense(model_size) #[tf.keras.layers.Dense(value_size) for _ in range(h)]
        self.wo = tf.keras.layers.Dense(model_size)
    
    def call(self, decoder_output, encoder_output, mask=None):
        
        
        query = self.wq(decoder_output)
        key = self.wk(encoder_output)
        value = self.wv(encoder_output)
        
        # Split for multihead attention
        batch_size = query.shape[0]
        query = tf.reshape(query, [batch_size, -1, self.h, self.key_size])
        query = tf.transpose(query, [0, 2, 1, 3])
        key = tf.reshape(key, [batch_size, -1, self.h, self.key_size])
        key = tf.transpose(key, [0, 2, 1, 3])
        value = tf.reshape(value, [batch_size, -1, self.h, self.key_size])
        value = tf.transpose(value, [0, 2, 1, 3])
        
        score = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(tf.dtypes.cast(self.key_size, dtype=tf.float32))
        
        if mask is not None:
            score *= mask
            score = tf.where(tf.equal(score, 0), tf.ones_like(score) * -1e9, score)
        
        alignment = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(alignment, value)
        context = tf.transpose(context, [0, 2, 1, 3])
        context = tf.reshape(context, [batch_size, -1, self.key_size * self.h])
        
        heads = self.wo(context)
        # heads has shape (batch, decoder_len, model_size)
        return heads

# Encoder Architecture

In [13]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, model_size, num_layers, h):
        super(Encoder, self).__init__()
        self.model_size = model_size
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, model_size)
        self.attention = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]

        self.attention_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]

        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(model_size) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]

    def call(self, sequence, padding_mask=None):
        embed_out = self.embedding(sequence)
        embed_out += pes[:sequence.shape[1], :]
        
        sub_in = embed_out
        
        for i in range(self.num_layers):
            sub_out = self.attention[i](sub_in, sub_in, padding_mask)
            sub_out = sub_in + sub_out
            sub_out = self.attention_norm[i](sub_out)
            
            ffn_in = sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_in + ffn_out
            ffn_out = self.ffn_norm[i](ffn_out)

            sub_in = ffn_out
            
        return ffn_out

# Decoder Architecture

In [14]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, model_size, num_layers, h):
        super(Decoder, self).__init__()
        self.model_size = model_size
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, model_size)
        self.attention_bot = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]
        self.attention_bot_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.attention_mid = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]
        self.attention_mid_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(model_size) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, sequence, encoder_output, padding_mask=None):
        # EMBEDDING AND POSITIONAL EMBEDDING
        embed_out = self.embedding(sequence)
        embed_out += pes[:sequence.shape[1], :]
        
        bot_sub_in = embed_out
        
        for i in range(self.num_layers):
            # BOTTOM MULTIHEAD SUB LAYER
            seq_len = bot_sub_in.shape[1]
            look_left_only_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
            bot_sub_out = self.attention_bot[i](bot_sub_in, bot_sub_in, look_left_only_mask)
            bot_sub_out = bot_sub_in + bot_sub_out
            bot_sub_out = self.attention_bot_norm[i](bot_sub_out)
            
            # MIDDLE MULTIHEAD SUB LAYER
            mid_sub_in = bot_sub_out

            mid_sub_out = self.attention_mid[i](mid_sub_in, encoder_output, padding_mask)
            mid_sub_out = mid_sub_out + mid_sub_in
            mid_sub_out = self.attention_mid_norm[i](mid_sub_out)

            # FFN
            ffn_in = mid_sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_out + ffn_in
            ffn_out = self.ffn_norm[i](ffn_out)

            bot_sub_in = ffn_out
        
        logits = self.dense(ffn_out)
            
        return logits

In [15]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)
def loss_func(targets, logits):
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [16]:
def predict(test_source_text=None):
    if test_source_text is None:
        test_source_text = data_english[np.random.choice(len(data_english))]
    print(test_source_text)
    test_source_seq = en_tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_output = encoder(tf.constant(test_source_seq))

    de_input = tf.constant([[fr_tokenizer.word_index['<sostoken>']]], dtype=tf.int64)

    out_words = []

    while True:
        de_output = decoder(de_input, en_output)
        new_word = tf.expand_dims(tf.argmax(de_output, -1)[:, -1], axis=1)
        out_words.append(fr_tokenizer.index_word[new_word.numpy()[0][0]])

        de_input = tf.concat((de_input, new_word), axis=-1)

        if out_words[-1] == '<eostoken>' or len(out_words) >= 10:
            break

    print(' '.join(out_words))

In [17]:
@tf.function
def train_step(source_seq, target_seq_in, target_seq_out):
    with tf.GradientTape() as tape:
        padding_mask = 1 - tf.cast(tf.equal(source_seq, 0), dtype=tf.float32)
        padding_mask = tf.expand_dims(padding_mask, axis=1)
        padding_mask = tf.expand_dims(padding_mask, axis=1)
        encoder_output = encoder(source_seq, padding_mask)
        
        decoder_output = decoder(target_seq_in, encoder_output, padding_mask)

        loss = loss_func(target_seq_out, decoder_output)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [18]:
NUM_LAYERS = 4
H = 4
in_vocab_size = len(en_tokenizer.word_index)+1
out_vocab_size = len(fr_tokenizer.word_index)+1
encoder = Encoder(in_vocab_size, MODEL_SIZE, NUM_LAYERS, H)
decoder = Decoder(out_vocab_size, MODEL_SIZE, NUM_LAYERS, H)

NUM_EPOCHS = 50

start_time = time.time()
for e in range(NUM_EPOCHS):
    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,target_seq_out)

    print('Epoch {} Loss {:.4f}'.format(
          e + 1, loss.numpy()))

    if (e + 1) % 10 == 0:
        end_time = time.time()
        print('Average elapsed time: {:.2f}s'.format((end_time - start_time) / (e + 1)))
        try:
            predict()
        except Exception as e:
            print(e)
            continue

Epoch 1 Loss 2.6879
Epoch 2 Loss 2.3960
Epoch 3 Loss 2.9521
Epoch 4 Loss 3.4382
Epoch 5 Loss 2.6638
Epoch 6 Loss 1.8867
Epoch 7 Loss 3.2869
Epoch 8 Loss 1.8722
Epoch 9 Loss 1.7625
Epoch 10 Loss 1.5821
Average elapsed time: 15.64s
tom was shot in the leg . 
[[10, 28, 493, 26, 8, 609, 1]]
tom etait dans tom tom tom . <eostoken>
Epoch 11 Loss 1.2029
Epoch 12 Loss 2.1495
Epoch 13 Loss 1.7730
Epoch 14 Loss 2.5962
Epoch 15 Loss 1.0851
Epoch 16 Loss 1.0686
Epoch 17 Loss 0.4954
Epoch 18 Loss 1.5528
Epoch 19 Loss 0.3746
Epoch 20 Loss 0.6439
Average elapsed time: 15.09s
when will we go home ? 
[[150, 82, 18, 37, 80, 5]]
quand allons nous nous rendre ? <eostoken>
Epoch 21 Loss 1.1378
Epoch 22 Loss 1.0409
Epoch 23 Loss 0.4193
Epoch 24 Loss 0.5797
Epoch 25 Loss 2.1686
Epoch 26 Loss 0.2490
Epoch 27 Loss 1.0539
Epoch 28 Loss 0.3042
Epoch 29 Loss 0.5682
Epoch 30 Loss 0.0222
Average elapsed time: 14.89s
i can t tell what it is . 
[[2, 17, 7, 79, 25, 11, 9, 1]]
je ne peux pas savoir quoi c est ce que
Ep

In [24]:
test_data =['I am a shy boy','I had to do it']
for t in test_data:
  pro = preprocess(t)
  predict(pro)

i am a shy boy
[[2, 88, 6, 855, 241]]
je suis un garcon timide . <eostoken>
i had to do it
[[2, 59, 4, 12, 11]]
il m a fallu le faire . <eostoken>
